# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data/Cities_Weather_Data.csv')
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,New Norfolk,-42.7826,147.0587,58.50,69,94,4.00,AU,1627433079
1,1,San Cristobal,7.7669,-72.2250,73.58,95,25,1.50,VE,1627433155
2,2,Nikolskoye,59.7035,30.7861,65.01,67,100,4.65,RU,1627433197
3,3,Rio Grande,-32.0350,-52.0986,41.95,62,5,20.40,BR,1627433604
4,4,Sibolga,1.7427,98.7792,78.42,86,52,2.24,ID,1627433605


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

FigureLayout={
    'width': '600x',
    'height': '600px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=FigureLayout, zoom_level=2, center=(0,0))

In [4]:
locations=df[["Lat","Lng"]].astype(float)
humidity=df["Humidity"].astype(float)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='600x'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Ideal Weather Conditions
# Max temp lower than 75 degrees but higher than 70
# Wind speed less than 5 mph
# Humidity less than 80% but higher than 40%
# Cloudiness less than 5%

ideal_df=df.loc[(df["Max Temp"]>=70)&(df["Max Temp"]<75)&(df["Wind Speed"]<5)&(df["Humidity"]<80)&(df["Humidity"]>40)&(df["Cloudiness"]<20)]

if len(ideal_df)>10:
    ideal_df=ideal_df.iloc[:10,:]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
180,180,Nizhniy Kuranakh,58.8353,125.4869,72.30,62,17,4.72,RU,1627433625
205,205,Verkhnevilyuysk,63.4453,120.3167,72.14,47,16,4.59,RU,1627433855
295,295,Kuytun,54.3419,101.5036,70.54,55,0,2.95,RU,1627433964
396,396,Brumado,-14.2036,-41.6653,72.41,56,4,4.59,BR,1627433824
403,403,Biograd na Moru,43.9433,15.4519,74.75,63,3,4.00,HR,1627434048
405,405,Boende,-0.2167,20.8667,70.11,45,17,2.17,CD,1627433728
442,442,Poya,-21.3500,165.1500,74.73,48,0,4.29,NC,1627434163
455,455,Weihe,32.9952,105.3247,73.89,61,0,4.56,CN,1627434181
471,471,Tommot,58.9564,126.2925,72.66,55,6,3.74,RU,1627434200
527,527,Villarrobledo,39.2699,-2.6012,73.78,49,0,3.20,ES,1627434274


### Hotel Map
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=ideal_df.copy()
hname=[]

target_locations=[]
target_radius=5000
target_query="hotel"
target_type="lodging"

x=range(len(ideal_df))

#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
for n in x:
    try:
        coord=f"{ideal_df.iloc[n,2]}, {ideal_df.iloc[n,3]}"
        search_params = {
            "location":coord,
            "type":target_type,
            "radius": target_radius,
            "key":g_key
        }
        response = requests.get(base_url, params=search_params).json()
        hname.append(response['results'][0]['name'])
    except:
        hname.append('No Hotels Within 5,000 Meters')
hotel_df["Hotel Name"]=hname


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
hotel_df
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='600x'))